<a href="https://colab.research.google.com/github/SaadBazaz/ChessRev/blob/main/ChessBot_AIProject2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chess Engine in Python

There are four main components to a chess engine:

- [x] Board representation
- [x] Move generation
- [x] Evaluation
- [ ] Search
- [ ] Optimization


# Concepts
## Move Generation
A version of chess so generic, that we can write a genetic algorithm to produce more fun versions of Chess.


e.g
n = can move "n" in this direction
a = attacks in this direction
b = blockable

example piece:

```
Piece:
team = white

piece_name = bishop

attack_dims = [
       [nba, -1,  nba,],
       [ -1,  x,   -1,],
       [nba, -1,  nba,],
]
```



```
Piece:
team = white

piece_name = castle

attack_dims = [
        [ -1, nba,   -1,],
        [nba,   x,  nba,],
        [ -1, nba,   -1,],
]
```


```
Piece:
team = white

piece_name = pawn

attack_dims = [
        [-1,  b,   a,],
        [-1,  x,  -1,],
        [-1, -1,  -1,],
]
```

```
Piece:
team = white

piece_name = horse

attack_dims = [
        [-1,  a,  b,   a,  -1,],
        [ a, -1,  b,  -1,   a,],
        [ b,  b,  x,   b,   b,],
        [ a, -1,  b,  -1,   a,],
        [-1,  a,  b,   a,  -1,],
]
```


## Imports
Standard and Third-party libraries used in this Chess Engine.

In [12]:
import random
import numpy as np
import copy
import itertools


# Board Representation & Move Generation
Constructing the board and pieces inside it.


## Possible Move Handles
Each Piece is expected to have a function which is called whenever moves are to be evaluated. They are defined here.


In [1]:

def pawn_possible_moves(self, board,x,y,team):
  possible_moves = []

  if team.name == "white":

    if (x-1 >= 0):
      if board.is_piece( (x-1, y) ) == False:
        possible_moves.append( (x-1, y) )
      if (x-2 >= 0):
        if board.is_piece( (x-2, y) ) == False and self.number_of_moves == 0:
          possible_moves.append( (x-2, y) )

      if (y+1 < 8):
        if board.is_piece( (x-1, y+1) ) == True and board[x][y].is_enemy(board[x-1][y+1]):
          possible_moves.append( (x-1, y+1) )
      if (y-1 >= 0):
        if board.is_piece( (x-1, y-1) ) == True and board[x][y].is_enemy(board[x-1][y-1]):
          possible_moves.append( (x-1, y-1) )


  elif team.name == "black":

    if (x+1 < 8):
      if board.is_piece( (x+1, y) ) == False:
        possible_moves.append( (x+1, y) )
      if (x+2 < 8):
        if board.is_piece( (x+2, y) ) == False and self.number_of_moves == 0:
          possible_moves.append( (x+2, y) )

      if (y+1 < 8):
        if board.is_piece( (x+1, y+1) ) == True and board[x][y].is_enemy(board[x+1][y+1]):
          possible_moves.append( (x+1, y+1) )
      if (y-1 >= 0):
        if board.is_piece( (x+1, y-1) ) == True and board[x][y].is_enemy(board[x+1][y-1]):
          possible_moves.append( (x+1, y-1) )


  return possible_moves


def rook_possible_moves(self, board,x,y,team, limit=-1):
  possible_moves = []

  i=1
  while( x-i >= 0 ):
    if board.is_piece( (x-i, y) ) == False:
      possible_moves.append( (x-i, y) )
    else:
      if board[x][y].is_enemy(board[x-i][y]):
        possible_moves.append( (x-i, y) )
      break
    if i==limit:
      break
    i+=1

  i=1
  while( x+i < 8 ):
    if board.is_piece( (x+i, y) ) == False:
      possible_moves.append( (x+i, y) )
    else:
      if board[x][y].is_enemy(board[x+i][y]):
        possible_moves.append( (x+i, y) )
      break
    if i==limit:
      break
    i+=1

  i=1
  while( y-i >= 0 ):
    if board.is_piece( (x, y-i) ) == False:
      possible_moves.append( (x, y-i) )
    else:
      if board[x][y].is_enemy(board[x][y-i]):
        possible_moves.append( (x, y-i) )
      break
    if i==limit:
      break
    i+=1

  i=1
  while( y+i < 8 ):
    if board.is_piece( (x, y+i) ) == False:
      possible_moves.append( (x, y+i) )
    else:
      if board[x][y].is_enemy(board[x][y+i]):
        possible_moves.append( (x, y+i) )
      break
    if i==limit:
      break
    i+=1

  return possible_moves


def bishop_possible_moves(self, board,x,y,team, limit=-1):
  possible_moves = []

  i=1
  while( x-i >= 0 and y-i >=0 ):
    if board.is_piece( (x-i, y-i) ) == False:
      possible_moves.append( (x-i, y-i) )
    else:
      if board[x][y].is_enemy(board[x-i][y-i]):
        possible_moves.append( (x-i, y-i) )
      break
    if i==limit:
      break
    i+=1

  i=1
  while( x+i < 8  and y+i < 8 ):
    if board.is_piece( (x+i, y+i) ) == False:
      possible_moves.append( (x+i, y+i) )
    else:
      if board[x][y].is_enemy(board[x+i][y+i]):
        possible_moves.append( (x+i, y+i) )
      break
    if i==limit:
      break
    i+=1

  i=1
  while( x-i >= 0 and y+i < 8 ):
    if board.is_piece( (x-i, y+i) ) == False:
      possible_moves.append( (x-i, y+i) )
    else:
      if board[x][y].is_enemy(board[x-i][y+i]):
        possible_moves.append( (x-i, y+i) )
      break
    if i==limit:
      break
    i+=1

  i=1 
  while( x+i < 8 and y-i >= 0 ):
    if board.is_piece( (x+i, y-i) ) == False:
      possible_moves.append( (x+i, y-i) )
    else:
      if board[x][y].is_enemy(board[x+i][y-i]):
        possible_moves.append( (x+i, y-i) )
      break
    if i==limit:
      break
    i+=1

  return possible_moves


def queen_possible_moves(self, board, x, y, team, limit=-1):
  return rook_possible_moves(self, board, x, y, team, limit) + bishop_possible_moves(self, board, x, y, team, limit)

def king_possible_moves(self, board, x, y, team, limit=1):
  return queen_possible_moves(self, board, x, y, team, limit)

def knight_possible_moves(self, board, x, y, team, limit=-1):
  possible_moves = []

  if ( y+2 < 8 ):
    if ( x-1 >= 0 ):
      if board.is_piece( (x-1, y+2) ) == False:
        possible_moves.append( (x-1, y+2) )
      else:
        if board[x][y].is_enemy(board[x-1][y+2]):
          possible_moves.append( (x-1, y+2) )

        
    if ( x+1 < 8):
      if board.is_piece( (x+1, y+2) ) == False:
        possible_moves.append( (x+1, y+2) )
      else:
        if board[x][y].is_enemy(board[x+1][y+2]):
          possible_moves.append( (x+1, y+2) )

  if ( y-2 >= 0 ):
    if ( x-1 >= 0 ):
      if board.is_piece( (x-1, y-2) ) == False:
        possible_moves.append( (x-1, y-2) )
      else:
        if board[x][y].is_enemy(board[x-1][y-2]):
          possible_moves.append( (x-1, y-2) )

    if ( x+1 < 8):
      if board.is_piece( (x+1, y-2) ) == False:
        possible_moves.append( (x+1, y-2) )
      else:
        if board[x][y].is_enemy(board[x+1][y-2]):
          possible_moves.append( (x+1, y-2) )

  if ( x+2 < 8 ):
    if ( y-1 >= 0 ):
      if board.is_piece( (x+2, y-1) ) == False:
        possible_moves.append( (x+2, y-1) )
      else:
        if board[x][y].is_enemy(board[x+2][y-1]):
          possible_moves.append( (x+2, y-1) )

    if ( y+1 < 8):
      if board.is_piece( (x+2, y+1) ) == False:
        possible_moves.append( (x+2, y+1) )
      else:
        if board[x][y].is_enemy(board[x+2][y+1]):
          possible_moves.append( (x+2, y+1) )

  if ( x-2 >= 0 ):
    if ( y-1 >= 0 ):
      if board.is_piece( (x-2, y-1) ) == False:
        possible_moves.append( (x-2, y-1) )
      else:
        if board[x][y].is_enemy(board[x-2][y-1]):
          possible_moves.append( (x-2, y-1) )

    if ( y+1 < 8):
      if board.is_piece( (x-2, y+1) ) == False:
        possible_moves.append( (x-2, y+1) )
      else:
        if board[x][y].is_enemy(board[x-2][y+1]):
          possible_moves.append( (x-2, y+1) )

  return possible_moves

## Models
Data models which hold or implement logic relevant to Chess.

In [2]:
class Piece:
  def __init__(self, name, FEN_name, possible_moves_handle, points):

    self.name = name
    self.acronym = FEN_name
    self.points = points
    
    self.possible_moves_handle = possible_moves_handle
    # self.attack_constraints = None

  
class GamePiece(Piece):
    def __init__(self, team, name, FEN_name, possible_moves_handle, points):
        self.team = team
        self.number_of_moves = 0
        super(GamePiece, self).__init__(name, FEN_name, possible_moves_handle, points)

    def is_enemy(self, that_piece):
      if self.team.name != that_piece.team.name:
        return True
      return False

    def __str__(self):
      return self.team.acronym + self.acronym


class Team:
  def __init__(self, name, human=False):
    self.name = name
    self.acronym = name[0]
    self.acquired_pieces = []
    self.points = 0
    self.human = human
    self.in_check = False

  def __str__(self):
    return acronym


class Board:
  def __init__(self, size, seed="rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR"):
    self.board  =   []
    self.seed   =   seed

    self.parse_fen(self.board, seed)

  def parse_fen(self, board, seed):
    new_row = []
    for letter in seed:
      if letter == "/":
        board.append(new_row)
        new_row = []
      elif letter.isnumeric():
        for i in range(0, int(letter)):
          new_row.append(  ""  )
      else:
        piece = pieces_lookup_dict.get(letter.lower(), None)
        team_name = team_names[1][0] if letter.islower() else team_names[0][0]
        team = teams_lookup_dict.get(team_name, None)

        game_piece = GamePiece(team, piece.name, piece.acronym, piece.possible_moves_handle, piece.points)
        if piece is not None:
          new_row.append(game_piece)
  
    board.append(new_row)
          

  def is_piece( self, loc ):
    x, y = loc
    if isinstance(board[x][y], GamePiece): return True
    return False
  

  def print_2D_board(self):
    for row in self.board:
      for cell in row:
        if cell == "":
          print ("-", "\t", end='')
        else:
          print (cell,"\t", end='')
      print ("\n")

  def check_possible_moves_of_cell(self, x, y):
    if self.is_piece ((x, y)):
      return (self.board[x][y].possible_moves_handle)(self.board[x][y], self, x, y, self.board[x][y].team)
    return None


  # overloaded for cell
  def check_possible_moves(self, cell):
    try:
      return (self.cell.possible_moves_handle)(self.board[x][y], self, x, y, cell.team)
    except:
      return None

  def check_possible_moves_of_board(self, team=None):
    possible_moves_dict = {}

    if team==None:
      for i, row in enumerate(self.board):
        for j, cell in enumerate(row):
          # print ("i, j =", i, j)
          possible_moves = self.check_possible_moves_of_cell(i, j)
          if possible_moves is not None:
            possible_moves_dict[(i,j)] = possible_moves
    
    else:    
      for i, row in enumerate(self.board):
        for j, cell in enumerate(row):
          if hasattr(cell, "team"):
            # print ("Is Equal? ", cell.team, team, " is", cell.team == team)
            if cell.team.name == team.name:
              possible_moves_dict[(i,j)] = self.check_possible_moves_of_cell(i, j)

    return possible_moves_dict

  def find_piece(self, acronym, team):
    for i, row in enumerate(self.board):
      for j, cell in enumerate(row):
        if self.is_piece( (i, j) ) and cell.acronym == acronym and cell.team.name == team.name:
          return (i, j)
    return None


  def evaluate_checks(self, team=None, return_instead_of_assign=False):
    
    if team==None:
      white_king = self.find_piece("k", teams_lookup_dict["w"])
      black_king = self.find_piece("k", teams_lookup_dict["b"])

      if white_king is None or black_king is None:
        return None

      possible_moves_dict = self.check_possible_moves_of_board()

      # print ("Possible moves are", possible_moves_dict)

      lists = possible_moves_dict.values()

      # print ("Possible moves values are", lists)
      combined = list(itertools.chain.from_iterable(lists))

      black_check = False
      white_check = False

      if white_king in combined:
        if return_instead_of_assign is False:
          white_check = True
          teams_lookup_dict["w"].in_check = True
        
      if black_king in combined:
        if return_instead_of_assign is False:
          black_check = True
          teams_lookup_dict["b"].in_check = True

      return white_check, black_check

  def move_to_old_loc(self,old_loc, new_loc):
        old_x, old_y = old_loc
        new_x, new_y = new_loc

        self.board[new_x][new_y] = copy.deepcopy(self.board[old_x][old_y])
        if old_loc == new_loc:
            return
        self.board[old_x][old_y] = ""

        self.board[new_x][new_y].number_of_moves -= 1
        return

  def move_piece (self, old_loc, new_loc):
    old_x, old_y = old_loc
    new_x, new_y = new_loc

    team = self.board[old_x][old_y].team

    # Check again, just in case
    possible_moves = self.check_possible_moves_of_cell(old_x, old_y)
    if possible_moves is None or new_loc not in possible_moves:
      return None


    if self.is_piece( new_loc ):
      # Add enemy piece to conquered pieces
      if self.board[old_x][old_y].is_enemy(self.board[new_x][new_y]):
        team.acquired_pieces.append( copy.deepcopy(self.board[new_x][new_y]) )
        team.points += self.board[new_x][new_y].points
      # Cannot move to location of an ally
      else: return False


    # Perform the move
    self.board[new_x][new_y] = copy.deepcopy(self.board[old_x][old_y])
    self.board[old_x][old_y] = ""

    self.board[new_x][new_y].number_of_moves += 1

    return True
    

  def __getitem__(self, x):
      return self.board[x]


## Initialization
Create different types of Pieces and Teams.

In [3]:
piece_names     =  ["Pawn","Rook","Knight","Bishop","King","Queen"]
piece_acronyms  =  ["p", "r", "n", "b", "k", "q"]
piece_possible_moves_handles  =  [
                    pawn_possible_moves,
                    rook_possible_moves,
                    knight_possible_moves,
                    bishop_possible_moves,
                    king_possible_moves,
                    queen_possible_moves
                    ]
piece_points    =   [10, 20, 40, 40, 900, 100000]

pieces_lookup_dict = {}


team_names = ["white","black"]
teams_lookup_dict = {}

# Create species of Pieces
for name, acronym, possible_moves_handle, points in zip(piece_names, piece_acronyms, piece_possible_moves_handles, piece_points):
  new_piece_type = Piece(name, acronym, possible_moves_handle, points)
  pieces_lookup_dict[acronym] = new_piece_type

# Create species of Teams
for name in team_names:
  new_team_type = Team(name)
  teams_lookup_dict[new_team_type.acronym] = new_team_type


In [4]:
board = Board(8, seed="rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR")
board.print_2D_board()
print(board.check_possible_moves_of_board(teams_lookup_dict["w"]))
print(board.check_possible_moves_of_cell(1,0))

br 	bn 	bb 	bq 	bk 	bb 	bn 	br 	

bp 	bp 	bp 	bp 	bp 	bp 	bp 	bp 	

- 	- 	- 	- 	- 	- 	- 	- 	

- 	- 	- 	- 	- 	- 	- 	- 	

- 	- 	- 	- 	- 	- 	- 	- 	

- 	- 	- 	- 	- 	- 	- 	- 	

wp 	wp 	wp 	wp 	wp 	wp 	wp 	wp 	

wr 	wn 	wb 	wq 	wk 	wb 	wn 	wr 	

{(6, 0): [(5, 0), (4, 0)], (6, 1): [(5, 1), (4, 1)], (6, 2): [(5, 2), (4, 2)], (6, 3): [(5, 3), (4, 3)], (6, 4): [(5, 4), (4, 4)], (6, 5): [(5, 5), (4, 5)], (6, 6): [(5, 6), (4, 6)], (6, 7): [(5, 7), (4, 7)], (7, 0): [], (7, 1): [(5, 0), (5, 2)], (7, 2): [], (7, 3): [], (7, 4): [], (7, 5): [], (7, 6): [(5, 5), (5, 7)], (7, 7): []}
[(2, 0), (3, 0)]


# Evaluation
Different heuristics/metrics we can use to evaluate the current game state.

In [5]:
def final_evaluation(board):
  pass

def material_evaluation(board,team):
  eval = 0
  for i, row in enumerate(board):
        for j, cell in enumerate(row):
          if cell != "":
            if cell.team.name == team:
              eval += cell.points
            else:
              eval -= cell.points


  print(eval)
  return eval

def positional_evaluation(board,team):
  pass



In [6]:
print(teams_lookup_dict["w"])
material_evaluation(board,teams_lookup_dict["w"])


q
-202360


-202360

In [7]:
# def ABpruning(board, depth, alpha, beta, team):
#     """
#     Returns the optimal action for the current player on the board.
#     """

#     if depth == 0:
#         return material_evaluation(board,team)

#     if team == "b":
#         possible_moves_for_team = board.check_possible_moves_of_board(teams_lookup_dict[team])
#         print("possible moves of b: ", possible_moves_for_team)
#         value = +99999999
#         for move,moves in possible_moves_for_team.items():
#           print('For move: ',move)
#           print('this move is: ',possible_moves_for_team[move])
#           if len(possible_moves_for_team[move]) == 0:
#               continue
#           board.move_piece(move, possible_moves_for_team[move][0])
#           board.print_2D_board()
#           value = min(value, ABpruning(board, depth + 1, alpha, beta, "w"))
#           if beta <= alpha:
#             print("breaking at 1")
#             break
#           return value
#     else:
#         possible_moves_for_team = board.check_possible_moves_of_board(teams_lookup_dict[team])
#         print("possible moves of w: ", possible_moves_for_team)
#         value = -99999999
#         for move in possible_moves_for_team:
#             if len(possible_moves_for_team[move]) == 0:
#                 continue
#             print('For move: ',move)
#             print('this move is: ',possible_moves_for_team[move])
#             board.move_piece(move, possible_moves_for_team[move][0])
#             board.print_2D_board()
#             value = max(value, ABpruning(board, depth + 1, alpha, beta, "b"))
#             alpha = max(alpha, value)
#             if alpha >= beta:
#               print("breaking at 2")
#               break
#             return value


In [21]:
best_move = []

# def ABpruning(board, depth, alpha, beta, team):
#     """
#     Returns the optimal action for the current player on the board.
#     """

#     if depth == 0:
#         return material_evaluation(board, team)

#     if team == "b":
#         possible_moves_for_team = board.check_possible_moves_of_board(teams_lookup_dict[team])
#         print("possible moves of b: ", possible_moves_for_team)
#         value = +99999999
#         for place, moves in possible_moves_for_team.items():
#             print('For move: ', place)
#             if len(moves) == 0:
#                 continue
#             prev_move = place
#             for move in moves:
#                 # temp solution to bring back the old place
#                 board.move_to_old_loc(prev_move, place)
#                 print('this move is: ', move)
#                 board.move_piece(place, move)
#                 board.print_2D_board()
#                 cur_value = ABpruning(copy.deepcopy(board), depth - 1, alpha, beta, "w")
#                 if (cur_value < value):
#                   value = cur_value
#                   best_move.append((place,move))
#                 beta = min(beta, cur_value)
#                 if beta <= alpha:
#                     print("breaking at 1")  
#                     break
#                 prev_move = move
#             return value
#     else:
#         possible_moves_for_team = board.check_possible_moves_of_board(teams_lookup_dict[team])
#         print("possible moves of w: ", possible_moves_for_team)
#         value = -99999999
#         for place, moves in possible_moves_for_team.items():
#             print('For move: ', place)
#             prev_move = place
#             for move in moves:
#                 # temp solution to bring back the old place
#                 board.move_to_old_loc(prev_move, place)
#                 print('this move is: ', move)
#                 board.move_piece(place, move)
#                 board.print_2D_board()
#                 cur_value = ABpruning(copy.deepcopy(board), depth - 1, alpha, beta, "b")
#                 if (cur_value > value):
#                   value = cur_value
#                   best_move.append((place,move))
#                 alpha = max(alpha, cur_value)
#                 if alpha >= beta:
#                     print("breaking at 2")
#                     break
#                 prev_move = move
#             return value

def ABpruning(board, depth, alpha, beta, team):
    """
    Returns the optimal action for the current player on the board.
    """

    if depth == 0:
        return material_evaluation(board, team)

    if team == "b":
        possible_moves_for_team = board.check_possible_moves_of_board(teams_lookup_dict[team])
        print("possible moves of b: ", possible_moves_for_team)
        value = +99999999
        for place, moves in possible_moves_for_team.items():
            print('For move: ', place)
            if len(moves) == 0:
                continue
            prev_move = place
            for move in moves:
                # temp solution to bring back the old place
                # board.move_to_old_loc(prev_move, place)
                print('this move is: ', move)
                board.move_piece(place, move)
                # board.print_2D_board()
                cur_value = ABpruning(copy.deepcopy(board), depth - 1, alpha, beta, "w")
                board.move_to_old_loc(move, place)
                if cur_value <= value:
                    value = cur_value
                    print('best move')
                    board.print_2D_board()
                    best_move.append((place, move))
                beta = min(beta, cur_value)
                if beta <= alpha:
                    print("breaking at 1")
                    break
                prev_move = move
        return value
    else:
        possible_moves_for_team = board.check_possible_moves_of_board(teams_lookup_dict[team])
        print("possible moves of w: ", possible_moves_for_team)
        value = -99999999
        for place, moves in possible_moves_for_team.items():
            print('For move: ', place)
            prev_move = place
            for move in moves:
                # temp solution to bring back the old place
                # board.move_to_old_loc(prev_move, place)
                print('this move is: ', move)
                board.move_piece(place, move)
                # board.print_2D_board()
                cur_value = ABpruning(copy.deepcopy(board), depth - 1, alpha, beta, "b")
                board.move_to_old_loc(move, place)
                if cur_value >= value:
                    value = cur_value
                    print('best move')
                    board.print_2D_board()
                    best_move.append((place, move))
                alpha = max(alpha, cur_value)
                if alpha >= beta:
                    print("breaking at 2")
                    break
                prev_move = move
        return value

# Game Loop
Here the logic for display, turns and endgames is implemented.

In [19]:
def main():

  white_first_turn = True
  option = input ("Is Black human? (Y/N)")
  if option == "Y":
    teams_lookup_dict["b"].human = True

  option = input ("Is White human? (Y/N)")
  if option == "Y":
    teams_lookup_dict["w"].human = True


  game_ongoing = True
  while (game_ongoing):

    board.print_2D_board()
    white_check, black_check = board.evaluate_checks()

    print ("White checked?", white_check)
    print ("Black checked?", black_check)

    is_human = False

    if white_first_turn: 
      print ("\n = = = = = = = White's turn!")
      curr_team = teams_lookup_dict["w"]
    else: 
      print("\n - - - - - - - Black's turn!")    
      curr_team = teams_lookup_dict["b"]

    is_human = curr_team.human
    possible_moves_for_team = board.check_possible_moves_of_board(curr_team)

    take_input = True

    if is_human:
      while (take_input):
        print ("Enter coordinates:")
        try:
          pos_x = int(input ("X:"))
          pos_y = int(input ("Y:"))
        except:
          print ("Invalid entry. Try again.")
          continue

        try:
          possible_moves_for_piece = possible_moves_for_team[(pos_x, pos_y)]
        except KeyError:
          print ("Could not find this piece. Does it exist and belong to your team?")
          continue

        print ("Possible moves are:", possible_moves_for_piece)

        index = int(input("Select an index (Enter -1 to select another piece):"))

        if (index == -1):
          print ("Choose another piece.")
          continue

        invalid_entry = True
        retry = True
        while (invalid_entry):
          try:
            move_to_make = possible_moves_for_piece[index]
            invalid_entry = False
            retry = False
          except:
            print ("Invalid entry. Try again.")
            index = int(input("Select an index (Enter -1 to select another piece):"))

            if (index == -1):
              print ("Choose another piece.")
              invalid_entry = False
              retry = True
              break


        if (retry):
          continue

        print ("You chose", move_to_make)

        if (board.move_piece ( (pos_x, pos_y) , (move_to_make) ) is False):
          print ("Illegal move. Your turn will be skipped.")
          continue
        
        take_input = False

    # Is Computer
    else:
      
      retry = True

      while retry:      
        # Random choices
        print('AB pruning is: ', ABpruning(copy.deepcopy(board), 2, -9999999999, 999999999, "b"))
        print(best_move[0])
        # pos_x, pos_y = random.choice(list(possible_moves_for_team))

        pos_x, pos_y = best_move[-1][0]
      
        possible_moves_for_piece = possible_moves_for_team[(pos_x, pos_y)]

        # If empty list of moves
        if not possible_moves_for_piece:
          continue

        # Else choose a random move
        # move_to_make = random.choice(possible_moves_for_piece)
        move_to_make = best_move[-1][1]

        if (board.move_piece ( (pos_x, pos_y) , (move_to_make) ) is False):
          print ("Illegal move. Your turn will be skipped.")
          continue
                
        retry = False

    white_first_turn = not white_first_turn


In [22]:
main()

Is Black human? (Y/N)N
Is White human? (Y/N)Y
br 	- 	bb 	bq 	bk 	bb 	bn 	br 	

bp 	bp 	bp 	bp 	bp 	wp 	bp 	bp 	

bn 	- 	- 	- 	- 	- 	- 	- 	

- 	wp 	- 	- 	- 	- 	- 	- 	

- 	- 	- 	- 	- 	- 	- 	- 	

- 	- 	- 	- 	- 	- 	- 	- 	

wp 	- 	wp 	wp 	wp 	wp 	- 	wp 	

wr 	wn 	wb 	wq 	wk 	wb 	wn 	wr 	

White checked? False
Black checked? True

 = = = = = = = White's turn!
Enter coordinates:
X:6
Y:2
Possible moves are: [(5, 2), (4, 2)]
Select an index (Enter -1 to select another piece):1
You chose (4, 2)
br 	- 	bb 	bq 	bk 	bb 	bn 	br 	

bp 	bp 	bp 	bp 	bp 	wp 	bp 	bp 	

bn 	- 	- 	- 	- 	- 	- 	- 	

- 	wp 	- 	- 	- 	- 	- 	- 	

- 	- 	wp 	- 	- 	- 	- 	- 	

- 	- 	- 	- 	- 	- 	- 	- 	

wp 	- 	- 	wp 	wp 	wp 	- 	wp 	

wr 	wn 	wb 	wq 	wk 	wb 	wn 	wr 	

White checked? False
Black checked? True

 - - - - - - - Black's turn!
possible moves of b:  {(0, 0): [(0, 1)], (0, 2): [], (0, 3): [], (0, 4): [(1, 5)], (0, 5): [], (0, 6): [(2, 5), (2, 7)], (0, 7): [], (1, 0): [(3, 0)], (1, 1): [(2, 1)], (1, 2): [(2, 2), (3, 2)], (1, 3

AttributeError: ignored


PseudoCode for AB pruning 
```
`# This is formatted as code`
function alphabeta(node, depth, α, β, maximizingPlayer) is.
    if depth = 0 or node is a terminal node then
        return the heuristic value of node
    if maximizingPlayer then
        value := −∞
        for each child of node do
            value := max(value, alphabeta(child, depth − 1, α, β, FALSE))
            α := max(α, value)
            if α ≥ β then
                break (* β cutoff *)
        return value
    else
        value := +∞
        for each child of node do
            value := min(value, alphabeta(child, depth − 1, α, β, TRUE))
            β := min(β, value)
            if β ≤ α then
                break (* α cutoff *)
        return value
```



In [23]:
# -*- coding: utf-8 -*-
"""ChessBot-AIProject2021

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/github/SaadBazaz/ChessRev/blob/main/ChessBot_AIProject2021.ipynb

# Chess Engine in Python

There are four main components to a chess engine:

- [x] Board representation
- [x] Move generation
- [x] Evaluation
- [ ] Search
- [ ] Optimization


# Concepts
## Move Generation
A version of chess so generic, that we can write a genetic algorithm to produce more fun versions of Chess.


e.g
n = can move "n" in this direction
a = attacks in this direction
b = blockable

example piece:

```
Piece:
team = white

piece_name = bishop

attack_dims = [
       [nba, -1,  nba,],
       [ -1,  x,   -1,],
       [nba, -1,  nba,],
]
```



```
Piece:
team = white

piece_name = castle

attack_dims = [
        [ -1, nba,   -1,],
        [nba,   x,  nba,],
        [ -1, nba,   -1,],
]
```


```
Piece:
team = white

piece_name = pawn

attack_dims = [
        [-1,  b,   a,],
        [-1,  x,  -1,],
        [-1, -1,  -1,],
]
```

```
Piece:
team = white

piece_name = horse

attack_dims = [
        [-1,  a,  b,   a,  -1,],
        [ a, -1,  b,  -1,   a,],
        [ b,  b,  x,   b,   b,],
        [ a, -1,  b,  -1,   a,],
        [-1,  a,  b,   a,  -1,],
]
```

## Imports
Standard and Third-party libraries used in this Chess Engine.
"""

import random
import numpy as np
import copy
import itertools

"""# Board Representation & Move Generation
Constructing the board and pieces inside it.

## Possible Move Handles
Each Piece is expected to have a function which is called whenever moves are to be evaluated. They are defined here.
"""


def pawn_possible_moves(self, board, x, y, team):
    possible_moves = []

    if team.name == "white":

        if (x - 1 >= 0):
            if board.is_piece((x - 1, y)) == False:
                possible_moves.append((x - 1, y))
            if (x - 2 >= 0):
                if board.is_piece((x - 2, y)) == False and self.number_of_moves == 0:
                    possible_moves.append((x - 2, y))

            if (y + 1 < 8):
                if board.is_piece((x - 1, y + 1)) == True and board[x][y].is_enemy(board[x - 1][y + 1]):
                    possible_moves.append((x - 1, y + 1))
            if (y - 1 >= 0):
                if board.is_piece((x - 1, y - 1)) == True and board[x][y].is_enemy(board[x - 1][y - 1]):
                    possible_moves.append((x - 1, y - 1))


    elif team.name == "black":

        if (x + 1 < 8):
            if board.is_piece((x + 1, y)) == False:
                possible_moves.append((x + 1, y))
            if (x + 2 < 8):
                if board.is_piece((x + 2, y)) == False and self.number_of_moves == 0:
                    possible_moves.append((x + 2, y))

            if (y + 1 < 8):
                if board.is_piece((x + 1, y + 1)) == True and board[x][y].is_enemy(board[x + 1][y + 1]):
                    possible_moves.append((x + 1, y + 1))
            if (y - 1 >= 0):
                if board.is_piece((x + 1, y - 1)) == True and board[x][y].is_enemy(board[x + 1][y - 1]):
                    possible_moves.append((x + 1, y - 1))

    return possible_moves


def rook_possible_moves(self, board, x, y, team, limit=-1):
    possible_moves = []

    i = 1
    while (x - i >= 0):
        if board.is_piece((x - i, y)) == False:
            possible_moves.append((x - i, y))
        else:
            if board[x][y].is_enemy(board[x - i][y]):
                possible_moves.append((x - i, y))
            break
        if i == limit:
            break
        i += 1

    i = 1
    while (x + i < 8):
        if board.is_piece((x + i, y)) == False:
            possible_moves.append((x + i, y))
        else:
            if board[x][y].is_enemy(board[x + i][y]):
                possible_moves.append((x + i, y))
            break
        if i == limit:
            break
        i += 1

    i = 1
    while (y - i >= 0):
        if board.is_piece((x, y - i)) == False:
            possible_moves.append((x, y - i))
        else:
            if board[x][y].is_enemy(board[x][y - i]):
                possible_moves.append((x, y - i))
            break
        if i == limit:
            break
        i += 1

    i = 1
    while (y + i < 8):
        if board.is_piece((x, y + i)) == False:
            possible_moves.append((x, y + i))
        else:
            if board[x][y].is_enemy(board[x][y + i]):
                possible_moves.append((x, y + i))
            break
        if i == limit:
            break
        i += 1

    return possible_moves


def bishop_possible_moves(self, board, x, y, team, limit=-1):
    possible_moves = []

    i = 1
    while (x - i >= 0 and y - i >= 0):
        if board.is_piece((x - i, y - i)) == False:
            possible_moves.append((x - i, y - i))
        else:
            if board[x][y].is_enemy(board[x - i][y - i]):
                possible_moves.append((x - i, y - i))
            break
        if i == limit:
            break
        i += 1

    i = 1
    while (x + i < 8 and y + i < 8):
        if board.is_piece((x + i, y + i)) == False:
            possible_moves.append((x + i, y + i))
        else:
            if board[x][y].is_enemy(board[x + i][y + i]):
                possible_moves.append((x + i, y + i))
            break
        if i == limit:
            break
        i += 1

    i = 1
    while (x - i >= 0 and y + i < 8):
        if board.is_piece((x - i, y + i)) == False:
            possible_moves.append((x - i, y + i))
        else:
            if board[x][y].is_enemy(board[x - i][y + i]):
                possible_moves.append((x - i, y + i))
            break
        if i == limit:
            break
        i += 1

    i = 1
    while (x + i < 8 and y - i >= 0):
        if board.is_piece((x + i, y - i)) == False:
            possible_moves.append((x + i, y - i))
        else:
            if board[x][y].is_enemy(board[x + i][y - i]):
                possible_moves.append((x + i, y - i))
            break
        if i == limit:
            break
        i += 1

    return possible_moves


def queen_possible_moves(self, board, x, y, team, limit=-1):
    return rook_possible_moves(self, board, x, y, team, limit) + bishop_possible_moves(self, board, x, y, team, limit)


def king_possible_moves(self, board, x, y, team, limit=1):
    return queen_possible_moves(self, board, x, y, team, limit)


def knight_possible_moves(self, board, x, y, team, limit=-1):
    possible_moves = []

    if (y + 2 < 8):
        if (x - 1 >= 0):
            if board.is_piece((x - 1, y + 2)) == False:
                possible_moves.append((x - 1, y + 2))
            else:
                if board[x][y].is_enemy(board[x - 1][y + 2]):
                    possible_moves.append((x - 1, y + 2))

        if (x + 1 < 8):
            if board.is_piece((x + 1, y + 2)) == False:
                possible_moves.append((x + 1, y + 2))
            else:
                if board[x][y].is_enemy(board[x + 1][y + 2]):
                    possible_moves.append((x + 1, y + 2))

    if (y - 2 >= 0):
        if (x - 1 >= 0):
            if board.is_piece((x - 1, y - 2)) == False:
                possible_moves.append((x - 1, y - 2))
            else:
                if board[x][y].is_enemy(board[x - 1][y - 2]):
                    possible_moves.append((x - 1, y - 2))

        if (x + 1 < 8):
            if board.is_piece((x + 1, y - 2)) == False:
                possible_moves.append((x + 1, y - 2))
            else:
                if board[x][y].is_enemy(board[x + 1][y - 2]):
                    possible_moves.append((x + 1, y - 2))

    if (x + 2 < 8):
        if (y - 1 >= 0):
            if board.is_piece((x + 2, y - 1)) == False:
                possible_moves.append((x + 2, y - 1))
            else:
                if board[x][y].is_enemy(board[x + 2][y - 1]):
                    possible_moves.append((x + 2, y - 1))

        if (y + 1 < 8):
            if board.is_piece((x + 2, y + 1)) == False:
                possible_moves.append((x + 2, y + 1))
            else:
                if board[x][y].is_enemy(board[x + 2][y + 1]):
                    possible_moves.append((x + 2, y + 1))

    if (x - 2 >= 0):
        if (y - 1 >= 0):
            if board.is_piece((x - 2, y - 1)) == False:
                possible_moves.append((x - 2, y - 1))
            else:
                if board[x][y].is_enemy(board[x - 2][y - 1]):
                    possible_moves.append((x - 2, y - 1))

        if (y + 1 < 8):
            if board.is_piece((x - 2, y + 1)) == False:
                possible_moves.append((x - 2, y + 1))
            else:
                if board[x][y].is_enemy(board[x - 2][y + 1]):
                    possible_moves.append((x - 2, y + 1))

    return possible_moves


"""## Models
Data models which hold or implement logic relevant to Chess.
"""


class Piece:
    def __init__(self, name, FEN_name, possible_moves_handle, points):
        self.name = name
        self.acronym = FEN_name
        self.points = points

        self.possible_moves_handle = possible_moves_handle
        # self.attack_constraints = None


class GamePiece(Piece):
    def __init__(self, team, name, FEN_name, possible_moves_handle, points):
        self.team = team
        self.number_of_moves = 0
        super(GamePiece, self).__init__(name, FEN_name, possible_moves_handle, points)

    def is_enemy(self, that_piece):
        if self.team.name != that_piece.team.name:
            return True
        return False

    def __str__(self):
        return self.team.acronym + self.acronym


class Team:
    def __init__(self, name, human=False):
        self.name = name
        self.acronym = name[0]
        self.acquired_pieces = []
        self.points = 0
        self.human = human
        self.in_check = False

    def __str__(self):
        return acronym


class Board:
    def __init__(self, size, seed="rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR"):
        self.board = []
        self.seed = seed

        self.parse_fen(self.board, seed)

    def parse_fen(self, board, seed):
        new_row = []
        for letter in seed:
            if letter == "/":
                board.append(new_row)
                new_row = []
            elif letter.isnumeric():
                for i in range(0, int(letter)):
                    new_row.append("")
            else:
                piece = pieces_lookup_dict.get(letter.lower(), None)
                team_name = team_names[1][0] if letter.islower() else team_names[0][0]
                team = teams_lookup_dict.get(team_name, None)

                game_piece = GamePiece(team, piece.name, piece.acronym, piece.possible_moves_handle, piece.points)
                if piece is not None:
                    new_row.append(game_piece)

        board.append(new_row)

    def is_piece(self, loc):
        x, y = loc
        if isinstance(board[x][y], GamePiece): return True
        return False

    def print_2D_board(self):
        for row in self.board:
            for cell in row:
                if cell == "":
                    print("-", "\t", end='')
                else:
                    print(cell, "\t", end='')
            print("\n")

    def check_possible_moves_of_cell(self, x, y):
        if self.is_piece((x, y)):
            return (self.board[x][y].possible_moves_handle)(self.board[x][y], self, x, y, self.board[x][y].team)
        return None

    # overloaded for cell
    def check_possible_moves(self, cell):
        try:
            return (self.cell.possible_moves_handle)(self.board[x][y], self, x, y, cell.team)
        except:
            return None

    def check_possible_moves_of_board(self, team=None):
        possible_moves_dict = {}

        if team == None:
            for i, row in enumerate(self.board):
                for j, cell in enumerate(row):
                    # print ("i, j =", i, j)
                    possible_moves = self.check_possible_moves_of_cell(i, j)
                    if possible_moves is not None:
                        possible_moves_dict[(i, j)] = possible_moves

        else:
            for i, row in enumerate(self.board):
                for j, cell in enumerate(row):
                    if hasattr(cell, "team"):
                        # print ("Is Equal? ", cell.team, team, " is", cell.team == team)
                        if cell.team.name == team.name:
                            possible_moves_dict[(i, j)] = self.check_possible_moves_of_cell(i, j)

        return possible_moves_dict

    def find_piece(self, acronym, team):
        for i, row in enumerate(self.board):
            for j, cell in enumerate(row):
                if self.is_piece((i, j)) and cell.acronym == acronym and cell.team.name == team.name:
                    return (i, j)
        return None

    def evaluate_checks(self, team=None, return_instead_of_assign=False):

        if team == None:
            white_king = self.find_piece("k", teams_lookup_dict["w"])
            black_king = self.find_piece("k", teams_lookup_dict["b"])

            if white_king is None or black_king is None:
                return None

            possible_moves_dict = self.check_possible_moves_of_board()

            # print ("Possible moves are", possible_moves_dict)

            lists = possible_moves_dict.values()

            # print ("Possible moves values are", lists)
            combined = list(itertools.chain.from_iterable(lists))

            black_check = False
            white_check = False

            if white_king in combined:
                if return_instead_of_assign is False:
                    white_check = True
                    teams_lookup_dict["w"].in_check = True

            if black_king in combined:
                if return_instead_of_assign is False:
                    black_check = True
                    teams_lookup_dict["b"].in_check = True

            return white_check, black_check

    def move_to_old_loc(self, old_loc, new_loc):
        old_x, old_y = old_loc
        new_x, new_y = new_loc

        self.board[new_x][new_y] = copy.deepcopy(self.board[old_x][old_y])
        if old_loc == new_loc:
            return
        self.board[old_x][old_y] = ""

        self.board[new_x][new_y].number_of_moves -= 1
        return

    def move_piece(self, old_loc, new_loc):
        old_x, old_y = old_loc
        new_x, new_y = new_loc

        team = self.board[old_x][old_y].team

        # Check again, just in case
        possible_moves = self.check_possible_moves_of_cell(old_x, old_y)
        if possible_moves is None or new_loc not in possible_moves:
            return None

        if self.is_piece(new_loc):
            # Add enemy piece to conquered pieces
            if self.board[old_x][old_y].is_enemy(self.board[new_x][new_y]):
                team.acquired_pieces.append(copy.deepcopy(self.board[new_x][new_y]))
                team.points += self.board[new_x][new_y].points
            # Cannot move to location of an ally
            else:
                return False

        # Perform the move
        self.board[new_x][new_y] = copy.deepcopy(self.board[old_x][old_y])
        self.board[old_x][old_y] = ""

        self.board[new_x][new_y].number_of_moves += 1

        return True

    def __getitem__(self, x):
        return self.board[x]


"""## Initialization
Create different types of Pieces and Teams.
"""

piece_names = ["Pawn", "Rook", "Knight", "Bishop", "King", "Queen"]
piece_acronyms = ["p", "r", "n", "b", "k", "q"]
piece_possible_moves_handles = [
    pawn_possible_moves,
    rook_possible_moves,
    knight_possible_moves,
    bishop_possible_moves,
    king_possible_moves,
    queen_possible_moves
]
piece_points = [10, 20, 40, 40, 900, 100000]

pieces_lookup_dict = {}

team_names = ["white", "black"]
teams_lookup_dict = {}

# Create species of Pieces
for name, acronym, possible_moves_handle, points in zip(piece_names, piece_acronyms, piece_possible_moves_handles,
                                                        piece_points):
    new_piece_type = Piece(name, acronym, possible_moves_handle, points)
    pieces_lookup_dict[acronym] = new_piece_type

# Create species of Teams
for name in team_names:
    new_team_type = Team(name)
    teams_lookup_dict[new_team_type.acronym] = new_team_type

board = Board(8, seed="rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR")
board.print_2D_board()
print(board.check_possible_moves_of_board(teams_lookup_dict["w"]))
print(board.check_possible_moves_of_cell(1, 0))

"""# Evaluation
Different heuristics/metrics we can use to evaluate the current game state.
"""


def final_evaluation(board):
    pass


def material_evaluation(board, team):
    eval = 0
    for i, row in enumerate(board):
        for j, cell in enumerate(row):
            if cell != "":
                if cell.team.name == team:
                    eval += cell.points
                else:
                    eval -= cell.points

    print(eval)
    return eval


def positional_evaluation(board, team):
    pass


print(teams_lookup_dict["w"])
material_evaluation(board, teams_lookup_dict["w"])

# def ABpruning(board, depth, alpha, beta, team):
#     """
#     Returns the optimal action for the current player on the board.
#     """

#     if depth == 0:
#         return material_evaluation(board,team)

#     if team == "b":
#         possible_moves_for_team = board.check_possible_moves_of_board(teams_lookup_dict[team])
#         print("possible moves of b: ", possible_moves_for_team)
#         value = +99999999
#         for move,moves in possible_moves_for_team.items():
#           print('For move: ',move)
#           print('this move is: ',possible_moves_for_team[move])
#           if len(possible_moves_for_team[move]) == 0:
#               continue
#           board.move_piece(move, possible_moves_for_team[move][0])
#           board.print_2D_board()
#           value = min(value, ABpruning(board, depth + 1, alpha, beta, "w"))
#           if beta <= alpha:
#             print("breaking at 1")
#             break
#           return value
#     else:
#         possible_moves_for_team = board.check_possible_moves_of_board(teams_lookup_dict[team])
#         print("possible moves of w: ", possible_moves_for_team)
#         value = -99999999
#         for move in possible_moves_for_team:
#             if len(possible_moves_for_team[move]) == 0:
#                 continue
#             print('For move: ',move)
#             print('this move is: ',possible_moves_for_team[move])
#             board.move_piece(move, possible_moves_for_team[move][0])
#             board.print_2D_board()
#             value = max(value, ABpruning(board, depth + 1, alpha, beta, "b"))
#             alpha = max(alpha, value)
#             if alpha >= beta:
#               print("breaking at 2")
#               break
#             return value

best_move = []


def ABpruning(board, depth, alpha, beta, team):
    """
    Returns the optimal action for the current player on the board.
    """

    if depth == 0:
        return material_evaluation(board, team)

    if team == "b":
        possible_moves_for_team = board.check_possible_moves_of_board(teams_lookup_dict[team])
        print("possible moves of b: ", possible_moves_for_team)
        value = +99999999
        for place, moves in possible_moves_for_team.items():
            print('For move: ', place)
            if len(moves) == 0:
                continue
            prev_move = place
            for move in moves:
                # temp solution to bring back the old place
                # board.move_to_old_loc(prev_move, place)
                print('this move is: ', move)
                board.move_piece(place, move)
                # board.print_2D_board()
                cur_value = ABpruning(copy.deepcopy(board), depth - 1, alpha, beta, "w")
                board.move_to_old_loc(move, place)
                if cur_value <= value:
                    value = cur_value
                    print('best move')
                    board.print_2D_board()
                    best_move.append((place, move))
                beta = min(beta, cur_value)
                if beta <= alpha:
                    print("breaking at 1")
                    break
                prev_move = move
        return value
    else:
        possible_moves_for_team = board.check_possible_moves_of_board(teams_lookup_dict[team])
        print("possible moves of w: ", possible_moves_for_team)
        value = -99999999
        for place, moves in possible_moves_for_team.items():
            print('For move: ', place)
            prev_move = place
            for move in moves:
                # temp solution to bring back the old place
                # board.move_to_old_loc(prev_move, place)
                print('this move is: ', move)
                board.move_piece(place, move)
                # board.print_2D_board()
                cur_value = ABpruning(copy.deepcopy(board), depth - 1, alpha, beta, "b")
                board.move_to_old_loc(move, place)
                if cur_value >= value:
                    value = cur_value
                    print('best move')
                    board.print_2D_board()
                    best_move.append((place, move))
                alpha = max(alpha, cur_value)
                if alpha >= beta:
                    print("breaking at 2")
                    break
                prev_move = move
        return value



"""# Game Loop
Here the logic for display, turns and endgames is implemented.
"""


def main():
    white_first_turn = True
    option = input("Is Black human? (Y/N)")
    if option == "Y":
        teams_lookup_dict["b"].human = True

    option = input("Is White human? (Y/N)")
    if option == "Y":
        teams_lookup_dict["w"].human = True

    game_ongoing = True
    while (game_ongoing):

        board.print_2D_board()
        white_check, black_check = board.evaluate_checks()

        print("White checked?", white_check)
        print("Black checked?", black_check)

        is_human = False

        if white_first_turn:
            print("\n = = = = = = = White's turn!")
            curr_team = teams_lookup_dict["w"]
        else:
            print("\n - - - - - - - Black's turn!")
            curr_team = teams_lookup_dict["b"]

        is_human = curr_team.human
        possible_moves_for_team = board.check_possible_moves_of_board(curr_team)

        take_input = True

        if is_human:
            while (take_input):
                print("Enter coordinates:")
                try:
                    pos_x = int(input("X:"))
                    pos_y = int(input("Y:"))
                except:
                    print("Invalid entry. Try again.")
                    continue

                try:
                    possible_moves_for_piece = possible_moves_for_team[(pos_x, pos_y)]
                except KeyError:
                    print("Could not find this piece. Does it exist and belong to your team?")
                    continue

                print("Possible moves are:", possible_moves_for_piece)

                index = int(input("Select an index (Enter -1 to select another piece):"))

                if (index == -1):
                    print("Choose another piece.")
                    continue

                invalid_entry = True
                retry = True
                while (invalid_entry):
                    try:
                        move_to_make = possible_moves_for_piece[index]
                        invalid_entry = False
                        retry = False
                    except:
                        print("Invalid entry. Try again.")
                        index = int(input("Select an index (Enter -1 to select another piece):"))

                        if (index == -1):
                            print("Choose another piece.")
                            invalid_entry = False
                            retry = True
                            break

                if (retry):
                    continue

                print("You chose", move_to_make)

                if (board.move_piece((pos_x, pos_y), (move_to_make)) is False):
                    print("Illegal move. Your turn will be skipped.")
                    continue

                take_input = False

        # Is Computer
        else:

            retry = True

            while retry:
                # Random choices
                print('AB pruning is: ', ABpruning(copy.deepcopy(board), 2, -9999999999, 999999999, "b"))
                print(best_move[0])
                # pos_x, pos_y = random.choice(list(possible_moves_for_team))

                pos_x, pos_y = best_move[-1][0]

                possible_moves_for_piece = possible_moves_for_team[(pos_x, pos_y)]

                # If empty list of moves
                if not possible_moves_for_piece:
                    continue

                # Else choose a random move
                # move_to_make = random.choice(possible_moves_for_piece)
                move_to_make = best_move[-1][1]

                if (board.move_piece((pos_x, pos_y), (move_to_make)) is False):
                    print("Illegal move. Your turn will be skipped.")
                    continue

                retry = False

        white_first_turn = not white_first_turn


main()

"""
PseudoCode for AB pruning 
```
`# This is formatted as code`
function alphabeta(node, depth, α, β, maximizingPlayer) is.
    if depth = 0 or node is a terminal node then
        return the heuristic value of node
    if maximizingPlayer then
        value := −∞
        for each child of node do
            value := max(value, alphabeta(child, depth − 1, α, β, FALSE))
            α := max(α, value)
            if α ≥ β then
                break (* β cutoff *)
        return value
    else
        value := +∞
        for each child of node do
            value := min(value, alphabeta(child, depth − 1, α, β, TRUE))
            β := min(β, value)
            if β ≤ α then
                break (* α cutoff *)
        return value
```

"""



Streaming output truncated to the last 5000 lines.

- 	wp 	- 	- 	- 	- 	- 	- 	

- 	- 	- 	wp 	- 	- 	wp 	- 	

wp 	- 	wp 	- 	wp 	wp 	- 	wp 	

wr 	wn 	wb 	wq 	wk 	wb 	wn 	wr 	

breaking at 2
For move:  (7, 7)
best move
br 	bn 	bb 	bq 	bk 	bb 	bn 	br 	

bp 	bp 	bp 	bp 	bp 	bp 	bp 	- 	

- 	- 	- 	- 	- 	- 	- 	- 	

- 	- 	- 	- 	- 	- 	- 	- 	

- 	wp 	- 	- 	- 	- 	- 	bp 	

- 	- 	- 	wp 	- 	- 	wp 	- 	

wp 	- 	wp 	- 	wp 	wp 	- 	wp 	

wr 	wn 	wb 	wq 	wk 	wb 	wn 	wr 	

this move is:  (3, 7)
possible moves of w:  {(4, 1): [(3, 1)], (5, 3): [(4, 3)], (5, 6): [(4, 6), (4, 7)], (6, 0): [(5, 0), (4, 0)], (6, 2): [(5, 2), (4, 2)], (6, 4): [(5, 4), (4, 4)], (6, 5): [(5, 5), (4, 5)], (6, 7): [(5, 7)], (7, 0): [], (7, 1): [(6, 3), (5, 0), (5, 2)], (7, 2): [(6, 1), (5, 0), (6, 3), (5, 4), (4, 5), (3, 6), (2, 7)], (7, 3): [(6, 3)], (7, 4): [(6, 3)], (7, 5): [(6, 6), (5, 7)], (7, 6): [(5, 5), (5, 7)], (7, 7): []}
For move:  (4, 1)
this move is:  (3, 1)
-202360
best move
br 	bn 	bb 	bq 	bk 	bb 	bn 	- 	

bp 	bp 	bp 	bp

AttributeError: ignored